I previously performed FPGA aware Neural architecutral search and found different models with different latency through evolutionary algorithm.

The obtained network from search process will be now retrained to improve accuracy. Also these models wt were intialized with the help of OFA!

In [1]:
import torch 
import torchvision
import os
import torch.nn as nn
from torchvision import transforms, datasets
import math
import time
from tqdm import tqdm
import shutil

In [2]:
cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    print('Using GPU.')
else:
    print('Using CPU.')

Using GPU.


In [3]:

batch_size=32

#I will use a susbset of imagenetval of 10k images 
if cuda_available:
    # path to the ImageNet dataset
    # link --> https://www.kaggle.com/datasets/titericz/imagenet1k-val
    
    imagenet_data_path = '/kaggle/input/imagenet1k-val/imagenet-val'

    # if 'imagenet_data_path' is empty, download a subset of ImageNet containing 2000 images (~250M) for test
    if not os.path.isdir(imagenet_data_path):
        print('%s is empty. Download a subset of ImageNet for test.' % imagenet_data_path)

    print('The ImageNet dataset files are ready.')
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')
    
    
  
if cuda_available:
    # The following function build the data transforms for test
    def build_val_transform(size):
        return transforms.Compose([
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])
    
    val_data = datasets.ImageFolder(
            root=os.path.join(imagenet_data_path),
            transform=build_val_transform(224)
        )
    

    val_loader = torch.utils.data.DataLoader(
        val_data,
        batch_size=batch_size,  
        shuffle = True,
        num_workers=4,  
        pin_memory=True,
        drop_last=False,
    )
    print('The ImageNet dataloader is ready. Size : {}'.format(len(val_loader)*batch_size))
else:
    data_loader = None
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

The ImageNet dataset files are ready.
The ImageNet dataloader is ready. Size : 50016


In [4]:

train_path = '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train'

train_data = datasets.ImageFolder(
            root= train_path,
            transform=build_val_transform(224)
        )

train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size, 
        shuffle = True,
        num_workers=4,  
        pin_memory=True,
        drop_last=False,
    )

print('The ImageNet train set is ready. Size : {}'.format(len(train_loader)*batch_size))

The ImageNet train set is ready. Size : 1281184


In [5]:
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = val_loader

dataset_sizes = {'train': len(train_loader)*32,
                'val': len(val_loader)*32}

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    #storing epoch data
    epoch_data = {
        'epoch': [],
        'train': {'loss': [], 'top1_acc': [], 'top5_acc': []},
        'val': {'loss': [], 'top1_acc': [], 'top5_acc': []}
    }
    
    # Create a temporary directory
    tempdir = '/kaggle/working/temp'
    os.makedirs(tempdir, exist_ok=True)
    best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

    torch.save(model.state_dict(), best_model_params_path)
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        epoch_data['epoch'].append(epoch+1)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            top1_corrects = 0
            top5_corrects = 0
            
            for inputs, labels in tqdm(dataloaders[phase], leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                
                # Calculate top-1 accuracy
                top1_corrects += torch.sum(preds == labels.data)
                
                # Calculate top-5 accuracy
                _, top5_preds = torch.topk(outputs, 5, dim=1)
                top5_corrects += torch.sum(top5_preds == labels.view(-1, 1))

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_top1_acc = top1_corrects.double() / dataset_sizes[phase]
            epoch_top5_acc = top5_corrects.double() / dataset_sizes[phase]
            
            epoch_data[phase]['loss'].append(epoch_loss)
            epoch_data[phase]['top1_acc'].append(epoch_top1_acc)
            epoch_data[phase]['top5_acc'].append(epoch_top5_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Top-1 Acc: {epoch_top1_acc:.4f} Top-5 Acc: {epoch_top5_acc:.4f}')

            if phase == 'val' and epoch_top1_acc > best_acc:
                best_acc = epoch_top1_acc
                best_top5 = epoch_top5_acc
                torch.save(model.state_dict(), best_model_params_path)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Top-1 Acc {best_acc:4f} /n Best val Top-5: {best_top5:4f}')

    model.load_state_dict(torch.load(best_model_params_path))

    # Clean up the temporary directory
    shutil.rmtree(tempdir)

    return model, epoch_data

In [7]:
!pip install ofa 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.1 MB/s eta 0:00:00


In [8]:
model = torch.load('/kaggle/input/searched-models-nas-fpga/models/moder_search_11.pth')

In [9]:
device = 'cuda' if cuda_available else 'cpu'

model = model.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.90)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.5)

In [10]:
torch.cuda.empty_cache()
model, epoch_data = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 1/5
----------


train Loss: 0.4665 Top-1 Acc: 0.8749 Top-5 Acc: 0.9790


val Loss: 0.9580 Top-1 Acc: 0.7636 Top-5 Acc: 0.9297

Epoch 2/5
----------


train Loss: 0.4341 Top-1 Acc: 0.8796 Top-5 Acc: 0.9805


val Loss: 0.9579 Top-1 Acc: 0.7627 Top-5 Acc: 0.9291

Epoch 3/5
----------


...

val Loss: 0.9628 Top-1 Acc: 0.7639 Top-5 Acc: 0.9295

Epoch 4/5
----------


train Loss: 0.4146 Top-1 Acc: 0.8843 Top-5 Acc: 0.9818


val Loss: 0.9655 Top-1 Acc: 0.7642 Top-5 Acc: 0.9294

Epoch 5/5
----------


train Loss: 0.4097 Top-1 Acc: 0.8854 Top-5 Acc: 0.9821


val Loss: 0.9582 Top-1 Acc: 0.7637 Top-5 Acc: 0.9299

Training complete in 503m 52s
Best val Top-1 Acc 0.764195 /n Best val Top-5: 0.929423


In [11]:
def plot_model_train(epoch_data, title = ' '):
    epochs = epoch_data['epoch']
    train_loss = epoch_data['train']['loss']
    val_loss = epoch_data['val']['loss']
    train_acc = epoch_data['train']['top1_acc']
    val_acc = epoch_data['val']['top1_acc']

    # Plotting the training and validation loss
    plt.figure(1)
    plt.figure(figsize=(10,6))
    plt.plot(epochs, train_loss, label='Training Loss', color='blue', linestyle='-', marker='o')
    plt.plot(epochs, val_loss, label='Validation Loss', color='red', linestyle='--', marker='s')
    plt.xlabel('Epoch', color='black')
    plt.ylabel('Loss', color='black')
    plt.title('Training and validation loss'+title, color='black')
    plt.legend(loc='upper right', facecolor='white', framealpha=1)
    plt.grid(color='gray', linestyle='--', linewidth=0.5)
    plt.xticks(range(1,21))
    plt.savefig('plot2.png')
    # Plotting the training and validation accuracy
    plt.figure(2)
    plt.figure(figsize=(10,6))
    plt.plot(epochs, train_acc, label='Training Accuracy', color='blue', linestyle='-', marker='^')
    plt.plot(epochs, val_acc, label='Validation Accuracy', color='red', linestyle='--', marker='d')
    plt.xlabel('Epoch', color='black')
    plt.ylabel('Accuracy', color='black')
    plt.title('Training and validation accuracy'+title, color='black')
    plt.legend(loc='lower right', facecolor='white', framealpha=1)
    plt.grid(color='gray', linestyle='--', linewidth=0.5)
    plt.xticks(range(1,21))
    
    n = random.randint(0,100)
    plt.savefig(f'plot{n}.png')

    plt.show()

In [12]:
plot_model_train(epoch_data, title = 'Retraining on Imagenet dataset (weight initialization from OFA)')

NameError: name 'plt' is not defined

In [ ]:
torch.save(model, 'model_retrained_search_11.pth')

In [ ]:
print(epoch_data)